In [1]:
#importing the required packages
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy,AUC
from tensorflow.keras.models import load_model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix,roc_auc_score,roc_curve
import itertools
import cv2
import scipy.integrate as integrate
import scipy
from tensorflow.keras.applications import VGG16
#from keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import ResNet50,ResNet101
import os

# Importing Images

In [ ]:
data_dir = 'train'  # path to the folder containing the dataset
test_dir = 'test'
img_width = 256  # width of the images
img_height = 256  # height of the images
batch_size = 32  # batch size for training

# Removing Corrupted Images

In [ ]:
def remove_corrupted_images():
    for folder in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                img = tf.keras.preprocessing.image.load_img(file_path, target_size=(img_width, img_height))
                img_arr = tf.keras.preprocessing.image.img_to_array(img)
            except:
                os.remove(file_path)

In [ ]:
remove_corrupted_images()

In [ ]:
def remove_corrupted_test():
    for folder in os.listdir(test_dir):
        folder_path = os.path.join(test_dir, folder)
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)
            try:
                img = tf.keras.preprocessing.image.load_img(file_path, target_size=(img_width, img_height))
                img_arr = tf.keras.preprocessing.image.img_to_array(img)
            except:
                os.remove(file_path)

In [ ]:
remove_corrupted_test()

# Spliting the datasets

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_set = train_datagen.flow_from_directory(data_dir,
                                              target_size=(img_width, img_height),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='training')
valid_set = train_datagen.flow_from_directory(data_dir, target_size=(img_width, img_height),
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              subset='validation')
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(test_dir, target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode='categorical',
                                            shuffle=False)

# Building the Deep Learning Classification model

In [ ]:
model = Sequential()
#CONV 1
model.add(Conv2D(filters= 32,kernel_size= (3, 3),strides=(3,3),padding='valid', activation='relu', input_shape=(img_width, img_height, 3))) #32 kernels/filters , kernel_size 3x3
model.add(MaxPooling2D(pool_size= (2, 2)))
model.add(BatchNormalization())


#CONV 2
model.add(Conv2D(filters=64,kernel_size= (3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005))) #increased the depth to 64
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(BatchNormalization())


#CONV 3
model.add(Conv2D(filters=64, kernel_size=(3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005))) #increased the depth to 64
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))


#CONV 4
model.add(Conv2D(filters=128,kernel_size= (3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(BatchNormalization())


#CONV 5
model.add(Conv2D(filters=128, kernel_size=(3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(BatchNormalization())

#CONV 6
model.add(Conv2D(filters=256, kernel_size=(3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005)))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2,2)))
model.add(BatchNormalization())

#CONV 7
model.add(Conv2D(filters=512, kernel_size=(3, 3),padding='same', activation='relu',strides=(1,1),kernel_regularizer=l2(0.0005)))
model.add(BatchNormalization())

model.add(Flatten())  #flattern, since there are too many dimensions; we only want a classification output

#CONV 8
model.add(Dense(units= 512, activation='relu')) #FC1: Fully connects to get all relevant data
model.add(Dropout(0.5))


model.add(Dense(units=3, activation='softmax')) #FC2: Outputs a softmax to squash the matrix into output probabilities for the 3 classes

# VGG16

In [ ]:
# Load pre-trained VGG16 model
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze pre-trained layers in VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Create new model with VGG16 as base
model = Sequential()
model.add(vgg16)

# Add custom layers for classification
model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=3, activation='softmax'))

# ResNet50

In [ ]:
# Load pre-trained ResNet50 model
resnet101 = ResNet101(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze pre-trained layers in ResNet50
for layer in resnet101.layers:
    layer.trainable = False

# Create new model with ResNet50 as base
model = Sequential()
model.add(resnet101)

# Add custom layers for classification
model.add(Flatten())
model.add(Dense(units=1000, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate = 0.00001)

In [ ]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Define a callback to save checkpoints during training
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath='checkpoints/cervical_cancer_screening_model-{epoch:02d}.h5',
                                                         monitor='val_loss',
                                                         verbose=1,
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         mode='auto')

In [ ]:
model.summary()

# Training the model

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist =  model.fit(train_set, epochs=30, validation_data=valid_set, callbacks=[tensorboard_callback,checkpoint_callback])

# Perfomance Plots

In [ ]:
history = model.history

for epoch in range(len(history.history['loss'])):
    loss = history.history['loss'][epoch]
    accuracy = history.history['accuracy'][epoch]
    print(f'Epoch {epoch}: loss = {loss}, accuracy = {accuracy}')

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# Evaluation

In [ ]:
# Load the weights from the last saved checkpoint
model.load_weights('checkpoints/cervical_cancer_screening_model-17.h5')

In [ ]:
# Get the predicted classes for the test set
y_pred = model.predict(test_set)
y_pred_classes = np.argmax(y_pred, axis=1)

# Create the confusion matrix
conf_matrix = confusion_matrix(test_set.classes, y_pred_classes, labels=None)

# Define the class labels
class_names = ['Negative', 'Positive','Suspected']

# Plot the confusion matrix
plt.imshow(conf_matrix, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)
fmt = 'd'
thresh = conf_matrix.max() / 2.
for i, j in itertools.product(range(conf_matrix.shape[0]), range(conf_matrix.shape[1])):
    plt.text(j, i, format(conf_matrix[i, j], fmt),
             horizontalalignment="center",
             color="white" if conf_matrix[i, j] > thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.savefig('confusion_matrix.pdf',format='pdf',bbox_inches='tight')
plt.show()

In [ ]:
# Calculate true positives, false positives, true negatives, and false negatives from the confusion matrix
tp = conf_matrix[1,1]
fp = conf_matrix[0,1]
tn = conf_matrix[0,0]
fn = conf_matrix[1,0]

# Calculate accuracy precision and recall
Accuracy = (tp+tn)/(tp+fp+tn+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
F_Score = 2*((precision*recall)/(precision+recall))

print('Accuracy:', np.round(Accuracy *100,4))
print('Precision:', np.round(precision*100,4))
print('Recall:', np.round(recall*100,4))
print('F-Score:', np.round(F_Score*100,4))

In [ ]:
# Calculate true positives, false positives, true negatives, and false negatives from the confusion matrix
tp = 195
fp = 46
tn = 179
fn = 32

# Calculate accuracy precision and recall
Accuracy = (tp+tn)/(tp+fp+tn+fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
F_Score = 2*((precision*recall)/(precision+recall))

print('Accuracy:', np.round(Accuracy *100,4))
print('Precision:', np.round(precision*100,4))
print('Recall:', np.round(recall*100,4))
print('F-Score:', np.round(F_Score*100,4))

In [ ]:
# Get the predicted probabilities
y_pred_prob = model.predict(test_set)

# Check the shape of the predicted probabilities
if y_pred_prob.ndim > 1 and y_pred_prob.shape[1] > 1:
    # Get the predicted probabilities for the positive class
    y_pred_prob = y_pred_prob[:, 1]

# Compute the AUC score
auc_score = roc_auc_score(y_true, y_pred_prob)

# Print the AUC score
print('AUC score:', auc_score)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_true, y_pred_prob)

In [ ]:
# plot ROC curve
plt.plot(fpr, tpr, label='ROC Curve (area = 0.96)')
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.savefig('AUC.pdf',format='pdf',bbox_inches='tight')
plt.show()

# Saving the model to json format

In [ ]:
model_json = model.to_json()
with open("model.json","w") as json_file:
    json_file.write(model_json)

# Testing the Model

# Saving the Model for deployment

In [ ]:
model.save(os.path.join('models','cervical_cancer_image_classifier.h5'))

In [ ]:
new_model = load_model('./models/cervical_cancer_image_classifier.h5')

In [ ]:
from tensorflow.keras.preprocessing import image
# Load the image and resize it to the required size for the model
img = image.load_img('data_set/Positive/4225.jpg', target_size=(256, 256))

# Convert the image to a numpy array
img_array = image.img_to_array(img)

# Expand the dimensions of the array to match the input shape of the model
img_array = np.expand_dims(img_array, axis=0)

# Normalize the pixel values to be between 0 and 1
img_array = img_array / 255.0

# Make a prediction using the model
preds = new_model.predict(img_array)

# Get the index of the class with the highest probability
class_idx = np.argmax(preds, axis=1)[0]

# Define a list of class labels
class_labels = ['Negative', 'Positive','Suspected']

# Get the corresponding class label for the predicted class index
class_label = class_labels[class_idx]

# Print the predicted class label
print('Predicted class:', class_label)
